In [1]:
interface Clickable {
    fun click()
}

class Button : Clickable {
    override fun click() = println("I was clicked")
}

fun main() {
    Button().click()
}

main()

I was clicked


- kotlinでは`override`修飾子が必須
- overrideするかrenameするまでコンパイルされない

- interfaceにはdefault実装がある

In [3]:
interface Clickable {
    fun click()
    fun showOff() = println("I'm clickable!")
}

class Button : Clickable {
    override fun click() = println("I was clicked")
}

fun main() {
    Button().click()
    Button().showOff() // default
}

main()

I was clicked
I'm clickable!


- `click()`は強制実装
- `showOff()`は必要とあれば

2つのインターフェイスに同じメソッドがある時にどちらが優先されるのか、それともされないのか
→コンパイルエラーがでます

```
Cannot infer visibility for 'showOff'. Specify it explicitly.
Class 'Button' must override 'showOff' because it inherits multiple interface methods for it.
```

In [ ]:
interface Clickable {
    fun click()
    fun showOff() = println("I'm clickable!")
}

interface Focusable {
    fun setFocus(b: Boolean) =
        println("I ${if (b) "got" else "lost"} focus.")

    fun showOff() = println("I'm focusable!")
}

class Button : Clickable, Focusable {
    override fun click() = println("I was clicked")
}

fun main() {
    Button().click()
    Button().showOff() // default
}

main()

次のように修正する

In [5]:
interface Clickable {
    fun click()
    fun showOff() = println("I'm clickable!")
}

interface Focusable {
    fun setFocus(b: Boolean) =
        println("I ${if (b) "got" else "lost"} focus.")

    fun showOff() = println("I'm focusable!")
}

class Button : Clickable, Focusable {
    override fun click() = println("I was clicked")

    override fun showOff() {
        super<Clickable>.showOff()
        super<Focusable>.showOff()
    }
}

fun main() {
    val button = Button()
    button.showOff()
    // I'm clickable!
    // I'm focusable!
    button.setFocus(true)
    // I got focus.
    button.click()
    // I was clicked.
}

main()

I'm clickable!
I'm focusable!
I got focus.
I was clicked


Open, final, and abstract modifiers: Final by default

Kotlinの全てのメソッドはデフォで*final*
    サブクラスは作れない
    基底クラスから任意のメソッドをoverrideできない

Javaは`final`修飾子をつかない限り、overrideやサブクラス作り放題だった

**壊れやすい基底クラス問題**
サブクラスがバカみたいに増えた時に、ちょっと基底クラスを変更すると、そいつらにどのような変更が起こるかわからん

*Effective Java* by Joshua Bloch
では*design and document for inheritance or elise prohibit it*を推奨している
継承に対して設計・ドキュメント化する、もしくは禁止
つまりJavaでは特別なことがない限りfinalをつけろ、じゃあKotlinではこの思想を受け継ぎ、
デフォルトでfinalにして、final修飾子を省こう





In [ ]:
interface Clickable {
    fun click()
    fun showOff() = println("I'm clickable!")
}

open class RichButton : Clickable {
    fun disable() { /* ... */ }
    open fun animate() { /* ... */ }
    override fun click() { /* ... */ }
}

class ThemeButton : RichButton() {
    override fun animate() { /* ... */ }
    override fun click() { /* ... */ }
    override fun showOff() { /* ... */ }
}

overrideできるやつらはデフォルトでopen

サブクラスでoverrideの継承止めたきゃfinalを明示的につける

In [ ]:
interface Clickable {
    fun click()
    fun showOff() = println("I'm clickable!")
}

open class RichButton : Clickable {
    fun disable() { /* ... */ }
    open fun animate() { /* ... */ }
    final override fun click() { /* ... */ }
}

class ThemeButton : RichButton() {
    override fun animate() { /* ... */ }
    override fun click() { /* ... */ }
    override fun showOff() { /* ... */ }
}

抽象クラス
    インスタンスを作れない
    抽象メンバーは常に`open`
    プロパティは常に`open`ではない。明示的に示す必要あり


In [ ]:
abstract class Animated {
    abstract val animationSpeed: Double
    val keyframes: Int = 20
    open val frames: Int = 60

    abstract fun animate()
    open fun stopAnimating() { /* ... */ }
    fun animateTwice() { /* ... */ }
}

`public`: どっからでも見える
`protected`: サブクラスで見える
`private`: クラスの中、もしくはトップレベル宣言の場合、ファイル内で見える
modifierをつけない場合には基本`public`

`internal`: モジュール内で見える
    モジュールとは一緒にコンパイルされる複数ファイルの単位

次の例はvisibilityを破る

In [1]:
internal open class TalkativeButton {
    private fun yell() = println("Hey!")
    protected fun whisper() = println("Let's talk!")
}

class class TalkativeSubButton : TalkativeButton() {
    fun giveSpeech() = println("give speech")
}

fun TalkativeButton.giveSpeech() { // エラー：publicメンバーはそのinternal recieverタイプTalkativeButtonを露出する
}


org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[1], line 6, column 6: Name expected

nested classはouter classのインスタンスにアクセスできない
なぜ重要か？

In [ ]:
import java.io.Serializable

interface State : Serializable

interface View {
    fun getCurrentState() : State
    fun restoreState(state: State) { /* ... */ }
}

In [ ]:
import java.io.Serializable

interface State : Serializable

interface View {
    fun getCurrentState() : State
    fun restoreState(state: State) { /* ... */ }
}

class Button : View {
    override fun getCurrentState(): State = ButtonState()
    override fun restoreState(state: State) { /* ... */ }
    class ButtonState : State { /* ... */ }
}

In [ ]:
class Outer {
    inner class Inner {
        fun getOuterReference(): Outer = this@Outer
    }
}

↓
```kotlin
class Mul(val left: Expr, val right: Expr): Expr
```
を追加したらwhenがexhaustiveにならなくなるよね！！↓
そして、コンパイラは追加しても、アラートを上げてくれない

In [ ]:
interface Expr
class Num(val value: Int): Expr
class Sum(val left: Expr, val right: Expr): Expr

fun eval(e: Expr): Int =
    when (e) {
        is Num -> e.value
        is Sum -> eval(e.right) + eval(e.left)
        else ->
            throw IllegalArgumentException("Unknown expression")
    }

sealed classでコンパイルタイムで問題に気づける
サブクラスの宣言は同じモジュール内に制限される
`interface`ではなく`sealed class`にする
`sealed class`は暗に抽象クラスである。明示的に`abstract`をつける必要はない

In [ ]:
sealed class Expr
class Num(val value: Int): Expr()
class Sum(val left: Expr, val right: Expr): Expr()

fun eval(e: Expr): Int =
    when (e) {
        is Num -> e.value
        is Sum -> eval(e.right) + eval(e.left)
        // Num, Sumしか宣言していないので、これ以外の可能性がない。→よってelseは必要ない。
    }

`Mul`を追加してみるとエラーが起こる

In [6]:
sealed class Expr
class Num(val value: Int): Expr()
class Sum(val left: Expr, val right: Expr): Expr()
class Mul(val left: Expr, val right: Expr): Expr()

fun eval(e: Expr): Int =
    when (e) {
        is Num -> e.value
        is Sum -> eval(e.right) + eval(e.left)
        // Num, Sumしか宣言していないので、これ以外の可能性がない。→よってelseは必要ない。
    }

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[6], line 7, column 5: 'when' expression must be exhaustive, add necessary 'else' branch

In [ ]:
sealed interface  Toggleable {
    fun toggle()
}

class LightSwitch: Toggleable {
    override fun toggle() = println("Lights!")
}

class Camera: Toggleable {
    override fun toggle() = println("Camera!")
}

OO Languageでは1つ以上のコンストラクタがある

primary constructor: 簡素な方法で初期化、クラスボディの外側で宣言
secondary constructor: クラスボディの中で宣言
initializer block: 追加の初期化ロジック

In [ ]:
class User(val nickname: String)

In [ ]:
// 最も明示的な方法で宣言
class User constructor(_nickname: String) {
    val nickname: String

    init {
        nickname = _nickname
    }
}

In [8]:
class User(
    val nickname: String,
    val isSubscribed: Boolean = true
)

fun main() {
    val alice = User("Alice")
    println(alice.isSubscribed)
    // true
    val bob = User("Bob", false)
    println(bob.isSubscribed)
    // false
    val carol = User("Carol", isSubscribed = false)
    println(carol.isSubscribed)
    // false
    val dave = User(nickname = "Dave", isSubscribed = true)
    println(dave.isSubscribed)
    // ture
}

main()

true
false
false
true


super classがconstructorを持っていれば、サブクラスも持たせる
vise verse

In [9]:
open class User(val nickname: String) { /* ... */ }
class SocialUser(nickname: String): User(nickname) { /* ... */ }

In [ ]:
open class Button
class RadioButton: Button()

private constructorはクラスの外でインスタンス化できない？？？
→ companion objectがそのようなconstructorを呼び出すのに良いらしい

In [ ]:
class Secret private constructor(private val agentName: String) {}

Tip:
overloadやデフォルト値を設定するためにsecondary constructorを宣言するな。
直接デフォルト値を指定せよ

↓以下はそれでもsecondary constructorを使う場面

In [ ]:
import java.net.URI

open class Downloader {
    constructor(uri: String?) {
        // some code
    }

    constructor(uri: URI?) {
        // some code
    }
}

これはprimary constructorがない。丸括弧がないので。2つのsecondary constructorがある。

secondary constructorが複数ある場合のsuper classを継承するときは、継承する側がもれなくsecondary constructorを宣言しなければならん。
super()が必要！！

In [ ]:
import java.net.URI

open class Downloader {
    constructor(url: String?) {
        // some code
    }

    constructor(uri: URI?) {
        // some code
    }
}

class MyDownloader: Downloader {
    constructor(url: String?): super(url) {
        // some code
    }

    constructor(uri: URI?): super(uri) {
        // some code
    }
}

自分のクラスから別のコンストラクタを呼び出せる

は？？？？

In [ ]:

import java.net.URI

open class Downloader {
    constructor(url: String?) {
        // some code
    }

    constructor(uri: URI?) {
        // some code
    }
}

class MyDownloader : Downloader {
    constructor(url: String?) : this(URI(url))
    constructor(uri: URI?) : super(uri)
}

In [3]:
import java.net.URI

open class Downloader {

    constructor(url: String?) {
        println("Downloader(String) constructor called")
        println("  url = $url")
    }

    constructor(uri: URI?) {
        println("Downloader(URI) constructor called")
        println("  uri = $uri")
    }
}

class MyDownloader : Downloader {

    constructor(url: String?) : this(URI(url)) {
        println("MyDownloader(String) constructor body")
    }

    constructor(uri: URI?) : super(uri) {
        println("MyDownloader(URI) constructor body")
    }
}

fun main() {
    println("=== String constructor ===")
    MyDownloader("https://example.com")

    println()
    println("=== URI constructor ===")
    MyDownloader(URI("https://example.com"))
}

main()


=== String constructor ===
Downloader(URI) constructor called
  uri = https://example.com
MyDownloader(URI) constructor body
MyDownloader(String) constructor body

=== URI constructor ===
Downloader(URI) constructor called
  uri = https://example.com
MyDownloader(URI) constructor body


Interfaceでproperty宣言を実装する

interfaceは値がbacking fieldとかgetterに保持されるかどうかは指定しない
→だから状態はもたない

In [5]:
interface User {
    val nickname: String
}

class PrivateUser(override val nickname: String): User

class SubscribingUser(val email: String): User {
    override val nickname: String
        get() = email.substringBefore('@') // custom getter
}

class SocialUser(val accountId: Int): User {
    override val nickname = getNameFromSocialNetwork(accountId)
}

fun getNameFromSocialNetwork(accountId: Int) =
    "kodee$accountId"

fun main() {
    println(PrivateUser("kodee").nickname)
    // kodee
    println(SubscribingUser("test@kotlinlang.org").nickname)
    // test
    println(SocialUser(123).nickname)
    // kodee123
}

main()

kodee
test
kodee123


In [ ]:
interface EmailUser {
    val email: String
    val nickname: String
        get() = email.substringBefore('@') // プロパティはバッキングフィールドを持っていないから、結果をアクセスのたびに毎回計算する
}

In [9]:
class User(val name: String) {
    var address: String = "unspecified"
        set(value: String) {
            println(
                """
                Address was changed for $name:
                "$field" -> "$value".
                """.trimIndent()
            )
            field = value
        }
}

fun main() {
    val user = User("Alice")
    user.address = "Christoph-Rapparini-Bogen 23"
    user.address = "Tatsuki Kodama"

    val user2 = User("Tatsuki")
    user2.address = "Shinohara"
}

main()

Address was changed for Alice:
"unspecified" -> "Christoph-Rapparini-Bogen 23".
Address was changed for Alice:
"Christoph-Rapparini-Bogen 23" -> "Tatsuki Kodama".
Address was changed for Tatsuki:
"unspecified" -> "Shinohara".


In [ ]:
class Person(var birthYear: Int) {
    var ageIn2050
        get() = 2050 - birthYear
        set(value) {
            birthYear = 2050 - value
        }
}

In [16]:
class LenghtCounter {
    var counter: Int = 0
        private set

    fun addWord(word: String) {
        counter += word.length
    }
}

fun main() {
    val lengthCounter = LenghtCounter()
    lengthCounter.addWord("Hi!")
    println(lengthCounter.counter)

    // lengthCounter.counter = 0
    // -> Cannot assign to 'counter': the setter is private in 'LenghtCounter'
}

main()

3


In [ ]:
class Customer(val name: String, val postralCode: Int) {
    override fun equals(other: Any?): Boolean {
        if (other == null || other !is Customer)
            return false
        return name == other.name && postralCode == other.postralCode
    }rride fun toString() = "Customer(name=$name, postralCode=$postralCode)

fun main() {
    val customer1 = Castomer("Alice", 324562)
    val customer2 = Castomer("Alice", 324562)
    println(customer1)
    println(custoftyhhyyyyyyyymer1.equals(customer2))
}

main()

委譲

In [ ]:
class DelegatingCollection<T>: Collection<T> {
    private val innerList = arrayListOf<T>()

    override val size: Int get() = innerList.size
    override fun isEmpty(): Boolean = innerList.isEmpty()
    override fun contains(element: T): Boolean = innerList.contains(element)
    override fun iterator(): Iterator<T> = innerList.iterator()
    override fun containsAll(elements: Collection<T>): Boolean =
        innerList.containsAll(elements)
}

In [ ]:
class DelegatingCollection<T>(
    innerList: Collection<T> = mutableListOf<T>()
): Collection<T> by innerList

In [3]:
class CountingSet<T>(
    private val innerSet: MutableCollection<T> = hashSetOf<T>()
): MutableCollection<T> by innerSet {
    var objectsAdded = 0

    override fun add(element: T): Boolean {
        objectsAdded++
        return innerSet.add(element)
    }

    override fun addAll(elements: Collection<T>): Boolean {
        objectsAdded += elements.size
        return innerSet.addAll(elements)
    }
}

fun main() {
    val cset = CountingSet<Int>()
    cset.addAll(listOf(1, 1, 2))
    println("Added ${cset.objectsAdded} objects, ${cset.size} uniques.")
}

main()

Added 3 objects, 2 uniques.


継承と委譲

In [4]:
open class SimpleList {
    open fun add(x: Int) {
        println("add $x")
    }

    open fun addAll(xs: List<Int>) {
        for (x in xs) {
            add(x)
        }
    }
}

class CountingList : SimpleList() {
    var count = 0

    override fun add(x: Int) {
        count++
        super.add(x)
    }

    override fun addAll(xs: List<Int>) {
        count += xs.size
        super.addAll(xs)
    }
}

fun main() {
    val c = CountingList()
    c.addAll(listOf(1, 2, 3))
    println(c.count)
}

main()


add 1
add 2
add 3
6


In [6]:
class CountingList2(
    private val inner: SimpleList = SimpleList()
) {
    var count = 0

    fun add(x: Int) {
        count++
        inner.add(x)
    }

    fun addAll(xs: List<Int>) {
        count += xs.size
        inner.addAll(xs)
    }
}

fun main() {
    val c = CountingList2()
    c.addAll(listOf(1, 2, 3))
    println(c.count)
}

main()

add 1
add 2
add 3
3


正当な委譲（byを使わなくて冗長）

In [7]:
class MyCollection(
    private val inner: MutableCollection<Int>
) : MutableCollection<Int> {

    override val size: Int
        get() = inner.size

    override fun add(element: Int): Boolean {
        println("add called")
        return inner.add(element)
    }

    override fun addAll(elements: Collection<Int>): Boolean {
        return inner.addAll(elements)
    }

    override fun iterator(): MutableIterator<Int> =
        inner.iterator()

    override fun clear() =
        inner.clear()

    override fun contains(element: Int): Boolean =
        inner.contains(element)

    override fun containsAll(elements: Collection<Int>): Boolean =
        inner.containsAll(elements)

    override fun isEmpty(): Boolean =
        inner.isEmpty()

    override fun remove(element: Int): Boolean =
        inner.remove(element)

    override fun removeAll(elements: Collection<Int>): Boolean =
        inner.removeAll(elements)

    override fun retainAll(elements: Collection<Int>): Boolean =
        inner.retainAll(elements)
}


In [ ]:
import java.util.Vector

object CaseInsensitiveFileComparator: Comparator<File> {
    override fun compare(file1: File, file2: File): Int {
        return file1.path.compareTo(file2.path, ignoreCase = true)
    }
}

fun main() {
    println(
        CaseInsenstiveFileComparator.compare(
            File("User/", File("/user"))
        )
    )
    // 0
}

In [3]:
data class Person(val name: String) {
    object NameComparator: Comparator<Person> {
        override fun compare(p1: Person, p2: Person): Int =
            p1.name.compareTo(p2.name)
    }
}

fun main() {
    val persons = listOf(Person("Bob"), Person("Alice"), Person("Charly"), Person("AB"))
    println(persons.sortedWith(Person.NameComparator))
    // [Person(name=Alice), Person(name=Bob)]
}

main()

[Person(name=AB), Person(name=Alice), Person(name=Bob), Person(name=Charly)]


companion object

In [5]:
class MyClass {
    companion object {
        fun callMe() {
            println("Companion object called")
        }
    }
}

fun main() {
    MyClass.callMe()

    val myObject = MyClass()
    // myObject.callMe() <-- クラスの外側からは見れない。
    // Error: Unresolved reference: callMe
}

main()

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[5], line 13, column 14: Unresolved reference: callMe

companion objectはprivateにアクセス可能
だからファクトリーパターンの理想的に候補になる

In [ ]:
class User {
    val nickname: String

    constructor(email: String) {
        nickname = email.substringBefore('@')
    }

    constructor(socialAccountId: Int) {
        nickname = getSocialNetworkName(socialAccountId)
    }
}

もう一つのやり方は

In [11]:
class User private constructor(val nickname: String) {

    companion object {
        fun newSubscribingUser(email: String) =
            User(email.substringBefore('@'))

        fun newSocialUser(accountId: Int) =
            User(getNameFromSocialNetwork(accountId)) // ソーシャルアカウントIDによって新しいユーザーを作成するファクトリーメソッド

        private fun getNameFromSocialNetwork(accoutId: Int): String { // mock function
            return when (accoutId) {
                1 -> "Tatsuki"
                2 -> "Takumi"
                3 -> "Fumiya"
                4 -> "Sora"
                else -> "Unknown person"
            }
        }
    }
}

fun main() {
    val subscribingUser = User.newSubscribingUser("bob@gmail.com")
    val socialUser = User.newSocialUser(4)
    println(subscribingUser.nickname)
    println(socialUser.nickname)
}

main()

bob
Sora


companion objectには名前をつけることができる
ただし、名前をつけようと付けまいと、どちらでもメンバーの呼び出しが可能

In [ ]:
class Person(val name: String) {
    companion object Loader {
        fun fromJSON(jsonText: String): Person = /* ... */
    }
}

fun main() {
    val person = Person.Loader.fromJSON("""{"name": "Dmitry"}""")
    println(person.name)
    // Dimitry
    val person2 = Person.fromJSON("""{"name": "Brent"}""")
    println(person2.name)
    // Brent
}

In [18]:
import kotlin.random.Random

fun main() {
    val chance = Random.nextInt(from = 0, until = 100)
    val coin = Random.Default.nextBoolean()
    println(chance)
    println(coin)
}

main()

87
true


In [ ]:
interface JSONFactory<T> {
    fun fromJSON(jsonText: String): T
}

class Person(val name: String) {
    companion object: JSONFactory<Person> {
        override fun fromJSON(jsonText: String): Person {
            TODO("Not yet implemented")
        }
    }
}

In [ ]:
// business locig module
class Person(val firstName: String, val lastName: String) {
    companion object {
    } // 空のコンパニオンオブジェクトを宣言しておく
}

// client/server communication module
fun Person.Companion.fromJSON(json: String): Person {
    /* ... */
}

val p = Person.fromJSON(json)

In [21]:
interface MouseListener {
    fun onEnter()
    fun onClick()
}

class Button()

fun main() {
    var clickCount = 0
    Button(object: MouseListener {
        override fun onEnter() { /* ...*/ }
        override fun onClick() {
            clickCount++
        }
    })
}

main()

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[21], line 10, column 12: Too many arguments for public constructor Button() defined in Line_21_jupyter.Button

Extra type safety without overhead: inline classes

In [ ]:
fun addExpense(expense: Int) {
    // save the expense as USD cent
}

addExpense(200) // Yen

YenやUSDが暗黙的で宣言的にして欲しい。
古典的な解決法は

In [22]:
class UsdCent(val amount: Int)

fun addExpense(expense: UsdCent) {
    // save the expense as USD cent
}

fun main() {
    addExpense(UsdCent(147))
}

In [ ]:
@JvmInline
class UsdCent(val amount: Int)

fun addExpense(expense: UsdCent) {
    // save the expense as USD cent
}

fun main() {
    addExpense(UsdCent(147))
}

単一の重要ではないオブジェクトをアホみたいに生成するのはGCの観点、無駄なメモリの観点から好ましくない。
よって@JvmInlineをアノテートすることで、これを回避できる。
このあのテートされたクラスをinline classという。
    inline classは1つのプロパティしか持てない。
    プライマリコンストラクタをもつ
    クラスヒエラルキーに属しない
    他のクラスを拡張しないし、拡張もされない
    インターフェイスまでは持って良い

In [23]:
interface PrettyPrintable {
    fun prettyPrint()
}

@JvmInline
value class UsdCent(val amount: Int): PrettyPrintable {
    val salesTax get() = amount * 0.06
    override fun prettyPrint() = println("${amount}¢")
}

fun main() {
    val expense = UsdCent(1_99)
    println(expense.salesTax)
    expense.prettyPrint()
}

main()

11.94
199¢
